# Кейс: ASR (Automatic speech recognition – автоматическое распознавание речи)

## Скачиваем необходимые библиотеки/фреймворки

In [ ]:
!pip install openai-whisper flask ffmpeg-python
!apt-get install ffmpeg
!pip install pyngrok

In [ ]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip

In [ ]:
!ngrok config add-authtoken 2qAfdhGGhliui7cmzQLZRwCzxRj_69brrS9x4NzVqhGovD72K

In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install pyannote.audio

In [ ]:
!pip install websockets

## Загружаем модели и создаем сайт на Flask

In [ ]:
import os
import whisper
from flask import Flask, request, jsonify, render_template, Response, send_file
from pyannote.audio import Pipeline
import librosa
import soundfile as sf
import threading
import time
import torch

app = Flask(__name__)

"""
Подключение токена HuggingFace
Для корректной работы приложения необходимо подключить свой токен HuggingFace.
Обратите внимание, что для использования моделей требуется наличие доступов (грантов) к следующим репозиториям:

1. Модель сегментации: https://huggingface.co/pyannote/segmentation
2. Модель для диаризации: https://huggingface.co/pyannote/speaker-diarization

Пожалуйста, убедитесь, что ваш токен предоставляет соответствующие права доступа к данным моделям.
Для подключения токена используется следующий код:
"""
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Загрузка моделей
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
whisper_model = whisper.load_model("tiny", device=device)
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=True).to(device)


def preprocess_audio(input_path, output_path):
    """Нормализация и ресемплинг аудио"""
    audio, sr = librosa.load(input_path, sr=16000)  # Ресемплинг до 16 кГц
    sf.write(output_path, audio, 16000, format='WAV')


def perform_diarization(filepath):
    """Диаризация аудио"""
    print(f"Performing diarization on {filepath}...")
    diarization = pipeline(filepath)
    speaker_segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        print(f"Speaker {speaker}: {turn.start:.1f}s - {turn.end:.1f}s")
        speaker_segments.append(f"{speaker}: {turn.start:.1f}s - {turn.end:.1f}s")
    return speaker_segments


def generate_srt(transcription):
    """Генерация субтитров в формате SRT"""
    segments = transcription["segments"]
    srt_content = ""
    for idx, segment in enumerate(segments):
        start = segment["start"]
        end = segment["end"]
        text = segment["text"]
        srt_content += f"{idx + 1}\n{format_time(start)} --> {format_time(end)}\n{text}\n\n"
    return srt_content


def format_time(seconds):
    """Форматирование времени для SRT"""
    millis = int((seconds - int(seconds)) * 1000)
    h, m, s = int(seconds // 3600), int((seconds % 3600) // 60), int(seconds % 60)
    return f"{h:02}:{m:02}:{s:02},{millis:03}"


def stream_transcription(filepath):
    """Постепенная транскрипция для отображения субтитров в реальном времени"""
    try:
        # Выполняем транскрипцию сразу
        result = whisper_model.transcribe(filepath, task="transcribe", language="ru", fp16=False)

        if "segments" in result:
            for idx, segment in enumerate(result["segments"]):
                start = segment['start']
                end = segment['end']
                text = segment['text']
                srt_content = f"{idx + 1}\n{format_time(start)} --> {format_time(end)}\n{text}\n\n"
                yield srt_content
                time.sleep(end - start)  # Синхронизация с реальным временем
        else:
            yield f"data: Ошибка: сегменты не найдены в результате.\n\n"
    except Exception as e:
        yield f"data: Ошибка: {str(e)}\n\n"


@app.route('/')
def index():
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>ASR App</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f4f4f9;
            }
            h1 {
                text-align: center;
            }
            form {
                max-width: 400px;
                margin: 0 auto;
                padding: 20px;
                background: white;
                border: 1px solid #ddd;
                border-radius: 8px;
            }
            label, select, input, button {
                display: block;
                width: 100%;
                margin-bottom: 10px;
            }
            button {
                background-color: #007BFF;
                color: white;
                border: none;
                padding: 10px;
                border-radius: 5px;
                cursor: pointer;
            }
            button:hover {
                background-color: #0056b3;
            }
            #subtitles {
                margin-top: 20px;
                max-width: 600px;
                margin: 20px auto;
                padding: 20px;
                background: #fff;
                border: 1px solid #ddd;
                border-radius: 8px;
                white-space: pre-wrap;
                font-family: monospace;
                overflow-y: auto;
                height: 200px;
            }
        </style>
    </head>
    <body>
        <h1>ASR-приложение</h1>
        <form action="/upload" method="post" enctype="multipart/form-data">
            <label for="method">Выберите метод обработки:</label>
            <select name="method">
                <option value="transcribe">Транскрипция</option>
                <option value="diarization">Диаризация</option>
                <option value="subtitles">Субтитры</option>
            </select>
            <input type="file" name="audio_file" accept="audio/*,video/*">
            <button type="submit">Загрузить</button>
        </form>
        <div id="subtitles"></div>
        <script>
            const eventSource = new EventSource('/stream_subtitles');
            const subtitlesDiv = document.getElementById('subtitles');
            const audioElement = document.getElementById('audio');

            eventSource.onmessage = function(event) {
                subtitlesDiv.textContent += event.data + "\n";
                subtitlesDiv.scrollTop = subtitlesDiv.scrollHeight;
            };

            fetch('/audio')
                .then(response => response.blob())
                .then(blob => {
                    const url = URL.createObjectURL(blob);
                    audioElement.src = url;
                })
                .catch(error => console.error('Ошибка загрузки аудио:', error));

            audioElement.addEventListener('play', () => {
                eventSource.onmessage = function(event) {
                    const [timeRange, text] = event.data.split(' ', 2);
                    const [start, end] = timeRange.split('-').map(parseFloat);

                    const currentTime = audioElement.currentTime;
                    if (currentTime >= start && currentTime <= end) {
                        subtitlesDiv.textContent += text + "\n";
                        subtitlesDiv.scrollTop = subtitlesDiv.scrollHeight;
                    }
                };
            });
        </script>
    </body>
    </html>
    """


@app.route('/upload', methods=['POST'])
def upload():
    if 'audio_file' not in request.files:
        return jsonify({"error": "Файл не найден"}), 400

    file = request.files['audio_file']
    if file.filename == '':
        return jsonify({"error": "Файл не выбран"}), 400

    method = request.form.get("method", "transcribe")
    filepath = os.path.join("uploads", file.filename)
    os.makedirs("uploads", exist_ok=True)
    file.save(filepath)

    preprocess_audio(filepath, filepath)

    if method == "transcribe":
        try:
            result = whisper_model.transcribe(filepath)
            os.remove(filepath)
            return jsonify({"text": result["text"]})
        except Exception as e:
            return jsonify({"error": str(e)}), 500

    elif method == "diarization":
        try:
            segments = perform_diarization(filepath)
            os.remove(filepath)
            return jsonify({"diarization": segments})
        except Exception as e:
            return jsonify({"error": str(e)}), 500

    elif method == "subtitles":
      try:
        return Response(stream_transcription(filepath), mimetype='text/event-stream')
      except Exception as e:
        return jsonify({"error": str(e)}), 500

    else:
        return jsonify({"error": "Неверный метод"}), 400


@app.route('/audio')
def audio():
    """Маршрут для потоковой передачи аудио"""
    filepath = os.path.join("uploads", os.listdir("uploads")[0])  # Берем первый файл в папке uploads
    return send_file(filepath, mimetype="audio/wav")

## Запускаем сайт

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print(f"Ваше приложение доступно по адресу: {public_url}\n")

# Запуск Flask
app.run(port=5000)